In [3]:
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras import datasets, layers, models
from sklearn.utils import shuffle

import Dataset_pb2


/Users/thymoterdoest/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/thymoterdoest/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/thymoterdoest/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/thymoterdoest/anaconda3/lib/python3.6/site-packages

In [4]:
BOARD_SIZE = 9;
CHANNELS = 2;

In [5]:
data_set = Dataset_pb2.DataSet()
f = open('../resources/dataset.bin', "rb")
data_set.ParseFromString(f.read())
f.close()

In [6]:
data = []
for data_instance in data_set.data:
    data.append(data_instance)
data = random.sample(data, len(data))

In [7]:
states = [];
values = [];
policies = [];
for data_instance in data:
    states.append(data_instance.state)
    values.append(data_instance.value)
    policies.append(data_instance.policy)

In [8]:
states = np.array(states)
values = np.array(values)
policies = np.array(policies)

In [16]:
states_3d = states.reshape((len(states), BOARD_SIZE, BOARD_SIZE, CHANNELS))
# change channels last to channels first format
states_3d_NCHW = np.moveaxis(states_3d, 3, 1)

In [17]:
states_3d_NCHW[0]

array([[[0, 0, 1, 0, 1, 0, 1, 0, 1],
        [0, 0, 1, 1, 0, 0, 1, 0, 0],
        [1, 1, 1, 0, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 1, 0, 1, 1],
        [0, 1, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0, 1, 1, 1, 1],
        [0, 1, 1, 0, 1, 0, 1, 1, 0],
        [1, 1, 1, 0, 0, 1, 1, 0, 1],
        [0, 1, 1, 1, 0, 0, 0, 0, 1]],

       [[1, 1, 0, 1, 0, 1, 0, 1, 0],
        [1, 1, 0, 0, 1, 1, 0, 1, 1],
        [0, 0, 0, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 1, 0, 1, 0, 0],
        [1, 0, 1, 1, 1, 0, 1, 0, 1],
        [1, 1, 0, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1, 0, 0, 1, 0],
        [1, 0, 0, 0, 1, 1, 1, 1, 0]]])

In [112]:
train_ds = tf.data.Dataset.from_tensor_slices((states_3d, values)).shuffle(10000).batch(32)

In [18]:
# Reserve 10,000 samples for validation
x_val = states_3d[-10000:]
y_val = values[-10000:]
x_train = states_3d[:-10000]
y_train = values[:-10000]

In [19]:
# class CNNModel(Model):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.conv1 = Conv2D(32, 3, padding='same', activation='relu')
#         self.pool1 = MaxPool2D((2,2))
#         self.conv2 = Conv2D(64, 3, padding='same', activation='relu')
#         self.pool2 = MaxPool2D((2,2))
#         self.flatten = Flatten()
#         self.d1 = Dense(512, activation='relu')
#         self.dropout1 = Dropout(0.4)
#         self.d2 = Dense(128, activation='relu')
#         self.dropout2 = Dropout(0.4)
#         self.d3 = Dense(43, activation='softmax')

#     def call(self, x):
#         x = self.conv1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.pool2(x)
#         x = self.flatten(x)
#         x = self.d1(x)
#         x = self.dropout1(x)
#         x = self.d2(x)
#         x = self.dropout2(x)
#         x = self.d3(x)
#         return x
# model = CNNModel()

model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(2, 9, 9)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.Conv2D(32, activation="relu", kernel_size=(3, 3),
                 input_shape=(BOARD_SIZE, BOARD_SIZE, CHANNELS),
                 data_format="channels_last",
                 padding='same'))
model.add(layers.Conv2D(32, activation="relu", kernel_size=(3, 3),
                 data_format="channels_last",
                 padding='same'))
model.add(layers.MaxPooling2D((2, 2), data_format="channels_last"))
model.add(layers.Conv2D(64 * 2, activation="relu", kernel_size=(3, 3),
                 data_format="channels_last",
                 padding='same'))
model.add(layers.Conv2D(64 * 2, activation="relu", kernel_size=(3, 3),
                 data_format="channels_last",
                 padding='same'))
model.add(layers.MaxPooling2D((2, 2), data_format="channels_last"))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='tanh'))

# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [10]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='mymodel_{epoch}.h5',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

In [21]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy','mae'])
# validation_data=(test_images, test_labels)

# Train the model by slicing the data into "batches"
# of size "batch_size", and repeatedly iterating over
# the entire dataset for a given number of "epochs"
print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    callbacks=callbacks,
                    validation_split=0.2)


Instructions for updating:
Use tf.cast instead.
# Fit model on training data
Train on 144259 samples, validate on 36065 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
144256/144259 [============================>.] - ETA: 0s - loss: 0.1195 - acc: 0.6956 - mean_absolute_error: 0.1420
Epoch 00001: val_loss improved from inf to 0.01412, saving model to mymodel_1.h5
144259/144259 [==============================] - 77s 531us/sample - loss: 0.1194 - acc: 0.6956 - mean_absolute_error: 0.1420 - val_loss: 0.0141 - val_acc: 0.7431 - val_mean_absolute_error: 0.0267
Epoch 2/3
144256/144259 [============================>.] - ETA: 0s - loss: 0.0182 - acc: 0.7400 - mean_absolute_error: 0.0239
Epoch 00002: val_loss improved from 0.01412 to 0.01282, saving model to mymodel_2.h5
144259/144259 [==============================] - 80s 551us/sample - loss: 0.0182 - acc: 0.7400 - mean_absolute_error: 0.0239 - val_loss: 0.0128 - val_acc: 0.7470 - val_mean_absolute_error: 0.0142
Epoch 3/3
1442

In [197]:
# The returned "history" object holds a record
# of the loss values and metric values during training
print('\nhistory dict:', history.history)

# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_val, y_val, batch_size=128)
print('test loss, test acc:', results)


history dict: {'loss': [0.0020016777825467313, 2.497330839640027e-07, 4.964430914252027e-08], 'acc': [0.7449258, 0.7454249, 0.7454249], 'mean_absolute_error': [0.0020172617, 3.7560294e-05, 1.7387269e-05], 'val_loss': [5.050072048655832e-07, 8.20101348738476e-08, 2.0804803639792533e-08], 'val_acc': [0.7474, 0.7474, 0.7474], 'val_mean_absolute_error': [5.1485138e-05, 2.0244155e-05, 1.0548238e-05]}

# Evaluate on test data
10000/10000 [==============================] - 1s 93us/sample - loss: 2.0805e-08 - acc: 0.7474 - mean_absolute_error: 1.0548e-05
test loss, test acc: [2.080480363244911e-08, 0.7474, 1.0548244e-05]


In [23]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_val[:10])
print('predictions shape:', predictions)


# Generate predictions for 3 samples
predictions shape: [[ 0.9999903 ]
 [ 0.9953068 ]
 [ 0.9989241 ]
 [-0.999984  ]
 [-0.020319  ]
 [-0.02061949]
 [-0.03048742]
 [-0.9998889 ]
 [-0.01936808]
 [ 1.        ]]


In [24]:
model.save('model.h5')  # creates a HDF5 file 'my_model.h5'